# 多量子比特系统可扩展性实验

## 实验概述

本实验专注于多量子比特系统的可扩展性分析，验证物理信息神经网络(PINN)在不同量子比特数下的性能表现。

In [92]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import sqrtm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# 设置随机种子确保可重复性
np.random.seed(42)
torch.manual_seed(42)

# 设置matplotlib样式
plt.style.use('default')
sns.set_palette("husl")

print("环境配置完成！")

环境配置完成！


In [93]:
class MultiQubitQuantumTools:
    """多量子比特系统工具类（支持自定义维度）"""
    
    @staticmethod
    def random_pure_state(dim=None, n_qubits=None):
        """生成随机纯态
        Args:
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        state = np.random.randn(dim) + 1j * np.random.randn(dim)
        return state / np.linalg.norm(state)
    
    @staticmethod
    def random_mixed_state(dim=None, n_qubits=None, purity=0.8):
        """生成随机混合态
        Args:
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
            purity: 纯度
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        # 生成随机纯态
        psi = MultiQubitQuantumTools.random_pure_state(dim=dim)
        rho = np.outer(psi, psi.conj())
        
        # 添加噪声以创建混合态
        identity = np.eye(dim, dtype=complex)
        rho_mixed = purity * rho + (1 - purity) * identity / dim
        return rho_mixed / np.trace(rho_mixed)
    
    @staticmethod
    def GHZ_state(dim=None, n_qubits=None):
        """生成GHZ态
        Args:
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        state = np.zeros(dim, dtype=complex)
        state[0] = 1/np.sqrt(2)
        state[-1] = 1/np.sqrt(2)
        return np.outer(state, state.conj())
    
    @staticmethod
    def W_state(dim=None, n_qubits=None):
        """生成W态
        Args:
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        
        # 对于任意维度，生成W-like态：前n个基态的均匀叠加
        state = np.zeros(dim, dtype=complex)
        n_terms = min(dim, 3)  # 使用前3个或所有基态
        for i in range(n_terms):
            state[i] = 1/np.sqrt(n_terms)
        return np.outer(state, state.conj())
    
    @staticmethod
    def pauli_operators():
        """返回单量子比特Pauli算符"""
        I = np.array([[1, 0], [0, 1]], dtype=complex)
        X = np.array([[0, 1], [1, 0]], dtype=complex)
        Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
        Z = np.array([[1, 0], [0, -1]], dtype=complex)
        return [I, X, Y, Z]
    
    @staticmethod
    def multiqubit_pauli_matrices(n_qubits):
        """生成多量子比特Pauli测量算符"""
        paulis = MultiQubitQuantumTools.pauli_operators()
        
        if n_qubits == 1:
            return paulis
        
        # 多量子比特情况：张量积
        multi_paulis = []
        for ops in np.ndindex((4,) * n_qubits):
            op = paulis[ops[0]]
            for i in range(1, n_qubits):
                op = np.kron(op, paulis[ops[i]])
            multi_paulis.append(op)
        
        return multi_paulis
    
    @staticmethod
    def generate_measurement_basis(dim):
        """生成d维系统的测量基（使用Gell-Mann矩阵的推广）
        Args:
            dim: 量子态维度
        Returns:
            测量基列表（d^2个算符）
        """
        bases = []
        
        # 单位矩阵
        I = np.eye(dim, dtype=complex)
        bases.append(I)
        
        # 对称矩阵（Gell-Mann类型）
        for j in range(dim):
            for k in range(j+1, dim):
                # 对称矩阵 |j><k| + |k><j|
                M = np.zeros((dim, dim), dtype=complex)
                M[j, k] = 1
                M[k, j] = 1
                bases.append(M)
        
        # 反对称矩阵（Gell-Mann类型）
        for j in range(dim):
            for k in range(j+1, dim):
                # 反对称矩阵 -i|j><k| + i|k><j|
                M = np.zeros((dim, dim), dtype=complex)
                M[j, k] = -1j
                M[k, j] = 1j
                bases.append(M)
        
        # 对角矩阵（Gell-Mann类型）
        for l in range(1, dim):
            M = np.zeros((dim, dim), dtype=complex)
            # 前l个对角元素为1，第l个对角元素为-l
            for m in range(l):
                M[m, m] = 1
            M[l, l] = -l
            # 归一化
            M = M / np.sqrt(l * (l + 1))
            bases.append(M)
        
        # 如果数量不够d^2，补充随机正交基
        while len(bases) < dim * dim:
            # 生成随机厄米矩阵
            A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
            M = (A + A.conj().T) / 2
            # 归一化
            M = M / np.linalg.norm(M)
            bases.append(M)
        
        # 只取前d^2个
        return bases[:dim * dim]
    
    @staticmethod
    def simulate_measurements(rho, n_measurements=1000, noise_level=0.01, dim=None):
        """模拟具有shot noise和系统噪声的量子测量过程
        Args:
            rho: 密度矩阵
            n_measurements: 测量次数
            noise_level: 噪声水平
            dim: 量子态维度（如果提供，优先使用；否则从rho.shape[0]推断）
        """
        if dim is None:
            dim = rho.shape[0]
        
        # 判断是否是2的幂次（标准量子比特系统）
        n_qubits = None
        if dim & (dim - 1) == 0:  # 检查是否是2的幂次
            n_qubits = int(np.log2(dim))
            # 使用Pauli测量基
            paulis = MultiQubitQuantumTools.multiqubit_pauli_matrices(n_qubits)
            measurement_bases = paulis
        else:
            # 使用Gell-Mann类型的测量基
            measurement_bases = MultiQubitQuantumTools.generate_measurement_basis(dim)
        
        frequencies = []
        for basis in measurement_bases:
            # 计算期望值
            expectation = np.real(np.trace(rho @ basis))
            
            # 归一化到[0,1]范围（对于非Pauli基，需要调整）
            if n_qubits is not None:
                # Pauli基：期望值在[-1,1]，映射到[0,1]
                prob = np.clip((expectation + 1) / 2, 0, 1)
            else:
                # 其他基：期望值可能在更大范围，需要归一化
                # 使用sigmoid函数映射
                prob = np.clip(1 / (1 + np.exp(-expectation)), 0, 1)
            
            # Shot noise：对每个测量设置随机的shot数
            jitter = max(1, int(n_measurements * 0.2))
            shots = max(10, int(n_measurements + np.random.randint(-jitter, jitter + 1)))
            shot_freq = np.random.binomial(shots, prob) / shots
            
            # 系统噪声：附加高斯噪声
            noisy_freq = shot_freq + np.random.normal(0, noise_level)
            noisy_freq = np.clip(noisy_freq, 0, 1)
            frequencies.append(noisy_freq)
        
        return np.array(frequencies)
    
    @staticmethod
    def fidelity(rho1, rho2):
        """计算量子态保真度"""
        # 确保输入是numpy数组
        if hasattr(rho1, 'cpu'):
            rho1 = rho1.cpu().numpy()
        if hasattr(rho2, 'cpu'):
            rho2 = rho2.cpu().numpy()
        
        sqrt_rho1 = sqrtm(rho1)
        fidelity_matrix = sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1)
        return np.real(np.trace(fidelity_matrix))**2
    
    @staticmethod
    def constraint_violation(rho):
        """计算物理约束违反程度"""
        # 确保输入是numpy数组
        if hasattr(rho, 'cpu'):
            rho = rho.cpu().numpy()
        
        # 厄米性违反
        hermiticity = np.linalg.norm(rho - rho.conj().T)
        
        # 迹归一化违反
        trace_violation = abs(np.trace(rho) - 1)
        
        # 正定性违反 (最小特征值)
        eigenvals = np.linalg.eigvals(rho)
        positivity_violation = max(0, -np.real(eigenvals).min())
        
        return hermiticity + trace_violation + positivity_violation
    
    @staticmethod
    def cholesky_to_density_matrix(alpha, dim=None, n_qubits=None):
        """从Cholesky参数构造密度矩阵
        Args:
            alpha: Cholesky参数向量
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        n_params = dim * (dim + 1) // 2
        
        # 确保alpha是numpy数组
        if hasattr(alpha, 'cpu'):
            alpha = alpha.detach().cpu().numpy()
        alpha = np.array(alpha, dtype=np.float64).flatten()
        
        if alpha.size < 2 * n_params:
            alpha = np.pad(alpha, (0, 2 * n_params - alpha.size), mode='constant')
        
        # 根据参数还原下三角矩阵
        L = np.zeros((dim, dim), dtype=complex)
        idx = 0
        for i in range(dim):
            for j in range(i+1):
                if i == j:
                    L[i, j] = abs(alpha[idx]) + 1e-9  # 对角元需为正
                    idx += 1
                else:
                    L[i, j] = alpha[idx] + 1j * alpha[idx + 1]
                    idx += 2
        
        rho = L @ L.conj().T
        rho = rho / np.trace(rho)
        return rho
    
    @staticmethod
    def density_to_cholesky_params(rho):
        """将密度矩阵转换为Cholesky参数向量"""
        if hasattr(rho, 'cpu'):
            rho = rho.detach().cpu().numpy()
        rho = np.array(rho, dtype=complex)
        rho = (rho + rho.conj().T) / 2  # 保证厄米
        dim = rho.shape[0]
        identity = np.eye(dim, dtype=complex)
        jitter = 1e-10
        for _ in range(6):
            try:
                L = np.linalg.cholesky(rho + jitter * identity)
                break
            except np.linalg.LinAlgError:
                jitter *= 10
        else:
            eigvals, eigvecs = np.linalg.eigh(rho)
            eigvals = np.clip(eigvals, 0, None)
            rho = eigvecs @ np.diag(eigvals) @ eigvecs.conj().T + jitter * identity
            L = np.linalg.cholesky(rho)
        L = np.tril(L)
        params = []
        for i in range(dim):
            for j in range(i+1):
                if i == j:
                    params.append(max(L[i, j].real, 1e-9))
                else:
                    params.append(L[i, j].real)
                    params.append(L[i, j].imag)
        return np.array(params, dtype=np.float32)
    
    @staticmethod
    def params_to_density_no_constraint(alpha, dim=None, n_qubits=None):
        """不施加Cholesky约束的直接参数->密度矩阵映射
        Args:
            alpha: 参数向量
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
        """
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        # 这个方法在多量子比特情况下更加复杂，此处简化实现
        if hasattr(alpha, 'cpu'):
            alpha = alpha.detach().cpu().numpy()
        alpha = np.array(alpha, dtype=np.float64).flatten()
        
        rho = np.zeros((dim, dim), dtype=complex)
        idx = 0
        for i in range(dim):
            for j in range(dim):
                if idx < len(alpha) - 1:
                    rho[i, j] = alpha[idx] + 1j * alpha[idx + 1]
                    idx += 2
        
        trace = np.trace(rho)
        if abs(trace) < 1e-9:
            trace = 1e-9
        rho = rho / trace
        return rho

In [94]:
class MultiQubitDataset(Dataset):
    """多量子比特量子态层析数据集（支持自定义维度）"""
    
    STATE_TYPES = ['GHZ', 'W', 'RandomPure', 'RandomMixed']
    
    def __init__(self, dim=None, n_qubits=None, n_samples=1000, n_measurements=1000, noise_level=0.01):
        """
        Args:
            dim: 量子态维度（如果提供，优先使用）
            n_qubits: 量子比特数（如果dim未提供，则使用2**n_qubits）
            n_samples: 样本数量
            n_measurements: 测量次数
            noise_level: 噪声水平
        """
        self.qt = MultiQubitQuantumTools()
        if dim is None:
            dim = 2**n_qubits if n_qubits is not None else 4
        self.dim = dim
        self.n_qubits = n_qubits
        
        # 生成描述字符串
        if n_qubits is not None and dim == 2**n_qubits:
            dim_str = f"{n_qubits}量子比特"
        else:
            dim_str = f"{dim}×{dim}"
        print(f"生成 {n_samples} 个 {dim_str} 量子态的数据集...")
        
        self.measurements = []
        self.true_states = []
        self.target_params = []
        self.state_types = []
        
        for i in tqdm(range(n_samples)):
            # 生成不同类型的量子态
            state_idx = i % 4
            state_type = self.STATE_TYPES[state_idx]
            if state_idx == 0:
                # GHZ态
                rho = self.qt.GHZ_state(dim=dim)
            elif state_idx == 1:
                # W态
                rho = self.qt.W_state(dim=dim)
            elif state_idx == 2:
                # 随机纯态
                psi = self.qt.random_pure_state(dim=dim)
                rho = np.outer(psi, psi.conj())
            else:
                # 随机混合态
                rho = self.qt.random_mixed_state(dim=dim)
            
            # 模拟测量
            freq = self.qt.simulate_measurements(rho, n_measurements, noise_level, dim=dim)
            
            self.measurements.append(freq)
            self.true_states.append(rho)
            self.state_types.append(state_type)
            
            # 真实Cholesky参数作为监督信号
            target_params = self.qt.density_to_cholesky_params(rho)
            self.target_params.append(target_params)
        
        self.measurements = np.array(self.measurements)
        self.true_states = np.array(self.true_states)
        self.target_params = np.array(self.target_params)
        self.state_types = np.array(self.state_types)
        
        print(f"数据集生成完成！")
        print(f"  测量数据形状: {self.measurements.shape}")
        print(f"  真实状态形状: {self.true_states.shape}")
        print(f"  目标参数形状: {self.target_params.shape}")
        print(f"  状态类型示例: {np.unique(self.state_types)}")
    
    def __len__(self):
        return len(self.measurements)
    
    def __getitem__(self, idx):
        return {
            'measurements': torch.FloatTensor(self.measurements[idx]),
            'true_state': self.true_states[idx],  # 保持为numpy数组
            'target_params': torch.FloatTensor(self.target_params[idx]),  # Cholesky参数
            'state_type': self.state_types[idx]
        }

In [95]:
class MultiQubitPINN(nn.Module):
    """多量子比特物理信息神经网络（改进版：完全可微分）"""
    
    def __init__(self, n_qubits, input_dim, hidden_dims=[128, 64], output_dim=None):
        super().__init__()
        
        self.n_qubits = n_qubits
        dim = 2**n_qubits
        self.dim = dim
        self.output_dim = dim * (dim + 1) // 2 * 2 if output_dim is None else output_dim
        
        # 网络架构
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, self.output_dim))
        
        self.network = nn.Sequential(*layers)
        
        # 物理约束参数 - 使用固定权重，不学习
        # 根据量子比特数自适应调整权重
        base_weight = 0.01  # 降低基础权重，避免压制MSE损失
        normalized_weight = base_weight / (dim ** 0.5)
        self.register_buffer('physics_weight', torch.tensor(normalized_weight))
        
        # 用于评估的旧方法（保持兼容性）
        self.qt = MultiQubitQuantumTools()
    
    def forward(self, x):
        return self.network(x)
    
    def cholesky_to_density_torch(self, alpha):
        """从Cholesky参数构造密度矩阵（PyTorch版本，保持梯度）"""
        batch_size = alpha.shape[0]
        dim = self.dim
        
        # 按照原始实现的格式解析参数
        # 对角元素：1个实数，非对角元素：2个实数（实部和虚部）
        n_triangular = dim * (dim + 1) // 2
        # 实际使用的参数数量：dim^2（但alpha可能有更多参数）
        n_params_used = dim * dim
        
        # 构建下三角矩阵L
        L_real = torch.zeros(batch_size, dim, dim, device=alpha.device, dtype=alpha.dtype)
        L_imag = torch.zeros(batch_size, dim, dim, device=alpha.device, dtype=alpha.dtype)
        
        idx = 0
        for i in range(dim):
            for j in range(i + 1):
                if idx >= n_params_used:
                    break
                if i == j:
                    # 对角元素：实数，需要为正
                    L_real[:, i, j] = torch.abs(alpha[:, idx]) + 1e-9
                    idx += 1
                else:
                    # 非对角元素：复数（实部和虚部）
                    if idx + 1 < alpha.shape[1]:
                        L_real[:, i, j] = alpha[:, idx]
                        L_imag[:, i, j] = alpha[:, idx + 1]
                        idx += 2
                    else:
                        break
        
        # 计算 rho = L @ L^dagger
        # L^dagger = L_real^T - 1j * L_imag^T
        L_real_T = L_real.transpose(-2, -1)
        L_imag_T = L_imag.transpose(-2, -1)
        
        # rho = L @ L^dagger = (L_real + 1j*L_imag) @ (L_real^T - 1j*L_imag^T)
        # 实部: L_real @ L_real_T + L_imag @ L_imag_T
        # 虚部: L_real @ L_imag_T - L_imag @ L_real_T
        rho_real = torch.bmm(L_real, L_real_T) + torch.bmm(L_imag, L_imag_T)
        rho_imag = torch.bmm(L_real, L_imag_T) - torch.bmm(L_imag, L_real_T)
        
        # 归一化：确保迹为1
        trace = torch.sum(rho_real[:, torch.arange(dim), torch.arange(dim)], dim=1, keepdim=True)
        trace = trace.unsqueeze(-1)  # [batch, 1, 1]
        rho_real = rho_real / (trace + 1e-9)
        rho_imag = rho_imag / (trace + 1e-9)
        
        return rho_real, rho_imag
    
    def compute_physics_loss_torch(self, rho_real, rho_imag):
        """计算物理约束损失（PyTorch版本，保持梯度）"""
        dim = self.dim
        batch_size = rho_real.shape[0]
        
        # 1. 厄米性违反：rho - rho^dagger 应该为0
        # rho^dagger 的实部 = rho_real^T, 虚部 = -rho_imag^T
        rho_real_T = rho_real.transpose(-2, -1)
        rho_imag_T = rho_imag.transpose(-2, -1)
        hermiticity_real = rho_real - rho_real_T
        hermiticity_imag = rho_imag + rho_imag_T  # 注意虚部的符号
        hermiticity_loss = torch.mean(hermiticity_real**2 + hermiticity_imag**2)
        
        # 2. 迹归一化违反：trace(rho) - 1 应该为0
        trace = torch.sum(rho_real[:, torch.arange(dim), torch.arange(dim)], dim=1)
        trace_violation = torch.mean((trace - 1.0)**2)
        
        # 3. 正定性违反：所有特征值应该 >= 0
        # 使用Gershgorin圆盘定理的近似，或者直接计算特征值
        # 为了可微分，我们使用：如果rho是正定的，则 rho + epsilon*I 的特征值都 > 0
        # 简化：使用 det(rho) >= 0 和 trace(rho) >= 0 作为近似
        # 更简单的方法：使用 rho 的最小对角元素作为正定性的近似
        diag_elements = rho_real[:, torch.arange(dim), torch.arange(dim)]
        min_diag = torch.min(diag_elements, dim=1)[0]
        positivity_loss = torch.mean(torch.clamp(-min_diag, min=0.0)**2)
        
        # 归一化损失
        normalization = dim ** 0.5
        total_loss = (hermiticity_loss + trace_violation + positivity_loss) / normalization
        
        return total_loss
    
    def predict_density_matrix(self, params):
        """从网络输出预测密度矩阵（用于评估，保持兼容性）"""
        if torch.is_tensor(params):
            alpha = params.detach().cpu().numpy()
        else:
            alpha = np.array(params)
        
        # 批量处理
        rhos = []
        for a in alpha:
            rho = self.qt.cholesky_to_density_matrix(a, self.n_qubits)
            rhos.append(rho)
        
        return np.array(rhos)
    
    def physics_loss(self, rho_pred):
        """计算物理约束损失（用于评估，保持兼容性）"""
        losses = []
        dim = 2**self.n_qubits
        for rho in rho_pred:
            cv = self.qt.constraint_violation(rho)
            normalized_cv = cv / (dim ** 0.5)
            losses.append(normalized_cv)
        return np.mean(losses)

In [96]:
class MultiQubitTrainer:
    """多量子比特模型训练器"""
    
    def __init__(self, model, device='cpu'):
        self.model = model.to(device)
        self.device = device
        self.qt = MultiQubitQuantumTools()
        
        # 确定量子比特数量
        n_qubits = 2  # 默认为2量子比特
        if hasattr(model, "n_qubits"):
            n_qubits = model.n_qubits
        
        # 优化器和损失函数
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.mse_loss = nn.MSELoss()
        
        # 训练历史
        self.train_losses = []
        self.val_losses = []
        self.constraint_violations = []
    
    def train_epoch(self, train_loader):
        """训练一个epoch"""
        self.model.train()
        epoch_loss = 0
        
        for batch in train_loader:
            measurements = batch['measurements'].to(self.device)
            target_params = batch['target_params'].to(self.device)
            
            self.optimizer.zero_grad()
            
            # 前向传播
            pred = self.model(measurements)
            
            # 计算损失：预测Cholesky参数与真实参数的MSE
            loss = self.mse_loss(pred, target_params)
            
            # 对于PINN，添加物理约束损失（使用可微分版本）
            if isinstance(self.model, MultiQubitPINN):
                # 使用PyTorch版本，保持梯度
                rho_real, rho_imag = self.model.cholesky_to_density_torch(pred)
                physics_loss = self.model.compute_physics_loss_torch(rho_real, rho_imag)
                
                # 使用固定权重（不学习）
                loss = loss + self.model.physics_weight * physics_loss
            
            # 反向传播
            loss.backward()
            self.optimizer.step()
            
            epoch_loss += loss.item()
        
        return epoch_loss / len(train_loader)
    
    def validate(self, val_loader):
        """验证模型"""
        self.model.eval()
        val_loss = 0
        constraint_violations = []
        
        with torch.no_grad():
            for batch in val_loader:
                measurements = batch['measurements'].to(self.device)
                target_params = batch['target_params'].to(self.device)
                
                pred = self.model(measurements)
                
                # 计算参数预测损失
                loss = self.mse_loss(pred, target_params)
                val_loss += loss.item()
                
                # 计算约束违反
                if isinstance(self.model, MultiQubitPINN):
                    rho_pred = self.model.predict_density_matrix(pred)
                    for rho in rho_pred:
                        cv = self.qt.constraint_violation(rho)
                        constraint_violations.append(cv)
        
        avg_cv = np.mean(constraint_violations) if constraint_violations else 0
        return val_loss / len(val_loader), avg_cv
    
    def train(self, train_loader, val_loader, epochs=50):
        """完整的训练过程"""
        print(f"开始训练 {type(self.model).__name__}...")
        
        for epoch in tqdm(range(epochs)):
            train_loss = self.train_epoch(train_loader)
            val_loss, cv = self.validate(val_loader)
            
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.constraint_violations.append(cv)
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, "
                      f"Val Loss: {val_loss:.4f}, CV: {cv:.6f}")
        
        return {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'constraint_violations': self.constraint_violations
        }

In [ ]:
# === 改进版 PINN 与训练器（覆盖上方定义，用于高维量子态） ===
class MultiQubitPINN(nn.Module):
    """多量子比特物理信息神经网络（强化表达 + 稳定物理约束）
    
    支持任意维度（包括非2的幂次维度）：
    - 如果提供dim参数，优先使用dim
    - 如果提供n_qubits参数，使用dim = 2**n_qubits
    - 如果两者都提供，优先使用dim
    """
    
    def __init__(self, n_qubits=None, input_dim=None, hidden_dims=None, output_dim=None, dropout=0.1, dim=None):
        super().__init__()
        if hidden_dims is None:
            hidden_dims = [512, 512, 256, 128]
        
        # 确定维度：优先使用dim参数，否则从n_qubits计算
        if dim is not None:
            self.dim = dim
            # 检查是否是2的幂次
            if dim & (dim - 1) == 0:
                self.n_qubits = int(np.log2(dim))
            else:
                self.n_qubits = None  # 非2的幂次维度
        elif n_qubits is not None:
            self.n_qubits = n_qubits
            self.dim = 2**n_qubits
        else:
            raise ValueError("必须提供n_qubits或dim参数之一")
        
        # 输出维度：dim*dim（Cholesky参数数量）
        self.output_dim = self.dim * self.dim if output_dim is None else output_dim
        
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.SiLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, self.output_dim))
        self.network = nn.Sequential(*layers)
        
        # 物理损失权重：根据维度自适应调整
        # 对于超高维度（>=10），使用固定大权重，确保物理约束占主导地位
        if self.dim >= 10:
            # 10×10及以上：使用固定大权重，不再归一化
            # 这样可以确保物理约束在高维度下仍然有效
            normalized_weight = 1.5e-3  # 固定权重，约为低维度的10-20倍
        elif self.dim >= 9:
            # 9×9：使用dim^1.0的归一化，保持物理约束的有效性
            base_weight = 3e-4
            normalized_weight = base_weight / self.dim
        else:
            # 低维度：使用dim^1.5的归一化
            base_weight = 2e-4
            normalized_weight = base_weight / (self.dim ** 1.5)
        self.register_buffer('physics_weight', torch.tensor(normalized_weight, dtype=torch.float32))
        self.qt = MultiQubitQuantumTools()
    
    def forward(self, x):
        return self.network(x)
    
    def cholesky_to_density_torch(self, alpha):
        batch_size = alpha.shape[0]
        dim = self.dim
        n_params_used = dim * dim
        L_real = torch.zeros(batch_size, dim, dim, device=alpha.device, dtype=alpha.dtype)
        L_imag = torch.zeros(batch_size, dim, dim, device=alpha.device, dtype=alpha.dtype)
        idx = 0
        for i in range(dim):
            for j in range(i + 1):
                if idx >= n_params_used:
                    break
                if i == j:
                    L_real[:, i, j] = torch.abs(alpha[:, idx]) + 1e-9
                    idx += 1
                else:
                    if idx + 1 < alpha.shape[1]:
                        L_real[:, i, j] = alpha[:, idx]
                        L_imag[:, i, j] = alpha[:, idx + 1]
                        idx += 2
                    else:
                        break
        L_real_T = L_real.transpose(-2, -1)
        L_imag_T = L_imag.transpose(-2, -1)
        rho_real = torch.bmm(L_real, L_real_T) + torch.bmm(L_imag, L_imag_T)
        rho_imag = torch.bmm(L_real, L_imag_T) - torch.bmm(L_imag, L_real_T)
        trace = torch.sum(rho_real[:, torch.arange(dim), torch.arange(dim)], dim=1, keepdim=True)
        trace = trace.unsqueeze(-1)
        rho_real = rho_real / (trace + 1e-9)
        rho_imag = rho_imag / (trace + 1e-9)
        return rho_real, rho_imag
    
    def compute_physics_loss_torch(self, rho_real, rho_imag):
        dim = self.dim
        rho_real_T = rho_real.transpose(-2, -1)
        rho_imag_T = rho_imag.transpose(-2, -1)
        hermiticity_real = rho_real - rho_real_T
        hermiticity_imag = rho_imag + rho_imag_T
        hermiticity_loss = torch.mean(hermiticity_real**2 + hermiticity_imag**2) / (dim ** 2)
        trace = torch.sum(rho_real[:, torch.arange(dim), torch.arange(dim)], dim=1)
        trace_violation = torch.mean((trace - 1.0)**2) / dim
        rho_complex = torch.complex(rho_real, rho_imag)
        eigvals = torch.linalg.eigvalsh(rho_complex)
        positivity_loss = torch.mean(torch.clamp(-eigvals, min=0.0)**2) / dim
        return hermiticity_loss + trace_violation + positivity_loss
    
    def predict_density_matrix(self, params):
        """从网络输出预测密度矩阵（用于评估，保持兼容性）"""
        if torch.is_tensor(params):
            alpha = params.detach().cpu().numpy()
        else:
            alpha = np.array(params)
        rhos = []
        for a in alpha:
            # 使用dim参数而不是n_qubits
            if self.n_qubits is not None:
                rho = self.qt.cholesky_to_density_matrix(a, n_qubits=self.n_qubits)
            else:
                rho = self.qt.cholesky_to_density_matrix(a, dim=self.dim)
            rhos.append(rho)
        return np.array(rhos)
    
    def physics_loss(self, rho_pred):
        """计算物理约束损失（用于评估，保持兼容性）"""
        losses = []
        dim = self.dim
        for rho in rho_pred:
            cv = self.qt.constraint_violation(rho)
            normalized_cv = cv / (dim ** 1.5)
            losses.append(normalized_cv)
        return np.mean(losses)


class MultiQubitTrainer:
    """多量子比特模型训练器（带学习率调度、梯度裁剪、物理损失warmup）"""
    
    def __init__(self, model, device='cpu', config=None):
        self.model = model.to(device)
        self.device = device
        self.qt = MultiQubitQuantumTools()
        self.config = config or {}
        self.current_epoch = 0
        self.learning_rates = []
        
        # 获取n_qubits和dim，处理None情况
        self.n_qubits = getattr(model, 'n_qubits', self.config.get('n_qubits', None))
        self.dim = getattr(model, 'dim', self.config.get('dim', None))
        
        # 确定基础学习率：对于高维PINN使用较小的学习率
        # 如果n_qubits为None（非2的幂次维度），使用dim来判断
        if isinstance(model, MultiQubitPINN):
            if self.n_qubits is not None and self.n_qubits >= 4:
                base_lr = 5e-4
            elif self.dim is not None and self.dim >= 16:
                base_lr = 5e-4  # 高维度（16×16及以上）使用较小学习率
            else:
                base_lr = 1e-3
        else:
            base_lr = 1e-3
        lr = self.config.get('lr', base_lr)
        betas = self.config.get('betas', (0.9, 0.999))
        weight_decay = self.config.get('weight_decay', 0.0)
        
        self.optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        if self.config.get('use_scheduler', True):
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                mode='min',
                factor=self.config.get('lr_factor', 0.5),
                patience=self.config.get('patience', 4),
                min_lr=self.config.get('min_lr', 1e-5)
            )
        else:
            self.scheduler = None
        
        self.grad_clip = self.config.get('grad_clip', 1.0)
        self.physics_warmup = self.config.get('physics_warmup', 0)
        self.mse_loss = nn.MSELoss()
        self.train_losses = []
        self.val_losses = []
        self.constraint_violations = []
    
    def train_epoch(self, train_loader):
        self.model.train()
        epoch_loss = 0
        for batch in train_loader:
            measurements = batch['measurements'].to(self.device)
            target_params = batch['target_params'].to(self.device)
            self.optimizer.zero_grad()
            pred = self.model(measurements)
            loss = self.mse_loss(pred, target_params)
            if isinstance(self.model, MultiQubitPINN):
                rho_real, rho_imag = self.model.cholesky_to_density_torch(pred)
                physics_loss = self.model.compute_physics_loss_torch(rho_real, rho_imag)
                warmup_factor = 1.0
                if self.physics_warmup:
                    warmup_factor = min(1.0, (self.current_epoch + 1) / self.physics_warmup)
                
                # 对于10×10等高维度，使用自适应权重调整
                # 确保物理损失在总损失中占合理比例
                if hasattr(self.model, 'dim') and self.model.dim >= 10:
                    # 计算MSE损失值（在detach后计算，避免影响梯度）
                    mse_loss_value = loss.detach().item()
                    physics_loss_value = physics_loss.detach().item()
                    # 如果物理损失相对MSE损失太小，动态增加权重
                    if mse_loss_value > 0 and physics_loss_value > 0:
                        # 计算未加权的物理损失相对于MSE的比例
                        raw_physics_ratio = physics_loss_value / mse_loss_value
                        # 如果原始物理损失太小，增加权重因子
                        if raw_physics_ratio < 0.05:  # 物理损失太小，大幅增加权重
                            adaptive_factor = 3.0
                        elif raw_physics_ratio < 0.1:
                            adaptive_factor = 2.0
                        elif raw_physics_ratio < 0.2:
                            adaptive_factor = 1.5
                        else:
                            adaptive_factor = 1.0
                        effective_weight = self.model.physics_weight * adaptive_factor
                    else:
                        effective_weight = self.model.physics_weight
                else:
                    effective_weight = self.model.physics_weight
                
                loss = loss + effective_weight * warmup_factor * physics_loss
            loss.backward()
            if self.grad_clip is not None:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
            self.optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / len(train_loader)
    
    def validate(self, val_loader):
        self.model.eval()
        val_loss = 0
        constraint_violations = []
        with torch.no_grad():
            for batch in val_loader:
                measurements = batch['measurements'].to(self.device)
                target_params = batch['target_params'].to(self.device)
                pred = self.model(measurements)
                loss = self.mse_loss(pred, target_params)
                val_loss += loss.item()
                if isinstance(self.model, MultiQubitPINN):
                    rho_pred = self.model.predict_density_matrix(pred)
                    for rho in rho_pred:
                        cv = self.qt.constraint_violation(rho)
                        constraint_violations.append(cv)
        avg_cv = np.mean(constraint_violations) if constraint_violations else 0.0
        return val_loss / len(val_loader), avg_cv
    
    def train(self, train_loader, val_loader, epochs=None):
        total_epochs = epochs or self.config.get('epochs', 50)
        print(f"开始训练 {type(self.model).__name__}，计划 {total_epochs} 轮...")
        for epoch in tqdm(range(total_epochs)):
            self.current_epoch = epoch
            train_loss = self.train_epoch(train_loader)
            val_loss, cv = self.validate(val_loader)
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.constraint_violations.append(cv)
            current_lr = self.optimizer.param_groups[0]['lr']
            self.learning_rates.append(current_lr)
            if self.scheduler is not None:
                self.scheduler.step(val_loss)
            log_interval = self.config.get('log_interval', 5)
            if (epoch + 1) % log_interval == 0 or epoch == 0:
                print(
                    f"Epoch {epoch+1}/{total_epochs}, "
                    f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, CV: {cv:.6f}, LR: {current_lr:.2e}"
                )
        return {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'constraint_violations': self.constraint_violations,
            'lrs': self.learning_rates
        }



In [98]:
def evaluate_model(model, test_loader, device='cpu'):
    """全面评估模型性能"""
    model.eval()
    qt = MultiQubitQuantumTools()
    
    # 确定维度和量子比特数量：优先从模型获取，否则从测试数据推断
    n_qubits = None
    dim = None
    
    # 从模型获取维度信息
    if hasattr(model, "dim"):
        dim = model.dim
        if hasattr(model, "n_qubits") and model.n_qubits is not None:
            n_qubits = model.n_qubits
        elif dim & (dim - 1) == 0:  # 检查是否是2的幂次
            n_qubits = int(np.log2(dim))
    elif hasattr(model, "n_qubits") and model.n_qubits is not None:
        n_qubits = model.n_qubits
        dim = 2 ** n_qubits
    else:
        # 从测试数据集的第一个样本推断维度
        sample = test_loader.dataset[0]
        true_state_sample = sample['true_state']
        if hasattr(true_state_sample, 'cpu'):
            true_state_sample = true_state_sample.cpu().numpy()
        dim = true_state_sample.shape[0]
        # 检查是否是2的幂次
        if dim & (dim - 1) == 0:
            n_qubits = int(np.log2(dim))
        else:
            n_qubits = None  # 非2的幂次维度
    
    fidelities = []
    mses = []
    constraint_violations = []
    violation_count = 0
    raw_total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            measurements = batch['measurements'].to(device)
            target_params = batch['target_params'].to(device)
            true_states = batch['true_state']
            
            # 预测
            pred = model(measurements)
            
            # 计算参数预测MSE
            mse = mean_squared_error(pred.cpu().numpy(), target_params.cpu().numpy())
            mses.append(mse)
            
            # 统计未加约束的违规率（仅对非PINN模型）
            if not isinstance(model, MultiQubitPINN):
                raw_params = pred.cpu().numpy()
                for params in raw_params:
                    # 使用dim或n_qubits参数
                    if dim is not None:
                        rho_raw = qt.params_to_density_no_constraint(params, dim=dim)
                    elif n_qubits is not None:
                        rho_raw = qt.params_to_density_no_constraint(params, n_qubits=n_qubits)
                    else:
                        continue  # 跳过无法处理的样本
                    cv_raw = qt.constraint_violation(rho_raw)
                    if cv_raw > 1e-3:
                        violation_count += 1
                raw_total += len(raw_params)
            
            # 对于所有模型，计算重建的密度矩阵质量
            # 根据模型类型选择适当的方法
            if isinstance(model, MultiQubitPINN):
                rho_pred = model.predict_density_matrix(pred)
            else:
                # 为Sequential模型添加预测密度矩阵的方法
                if torch.is_tensor(pred):
                    alpha = pred.detach().cpu().numpy()
                else:
                    alpha = np.array(pred)
                
                # 批量处理：为每个样本从true_state获取实际维度
                rhos = []
                for idx, a in enumerate(alpha):
                    # 从对应的true_state获取实际维度
                    rho_true_sample = true_states[idx]
                    if hasattr(rho_true_sample, 'cpu'):
                        rho_true_sample = rho_true_sample.cpu().numpy()
                    target_dim = rho_true_sample.shape[0]
                    
                    # 使用dim或n_qubits参数
                    if target_dim & (target_dim - 1) == 0:
                        # 是2的幂次，可以使用n_qubits
                        target_n_qubits = int(np.log2(target_dim))
                        rho = qt.cholesky_to_density_matrix(a, n_qubits=target_n_qubits)
                    else:
                        # 非2的幂次，使用dim参数
                        rho = qt.cholesky_to_density_matrix(a, dim=target_dim)
                    rhos.append(rho)
                rho_pred = np.array(rhos)
            
            for i, rho_p in enumerate(rho_pred):
                # 确保rho_true是numpy数组
                rho_true = true_states[i]
                if hasattr(rho_true, 'cpu'):
                    rho_true = rho_true.cpu().numpy()
                
                # 维度检查：确保rho_pred和rho_true的维度匹配
                if rho_p.shape != rho_true.shape:
                    print(f"警告：维度不匹配！rho_pred.shape={rho_p.shape}, rho_true.shape={rho_true.shape}，跳过此样本")
                    continue
                
                # 计算保真度
                fid = qt.fidelity(rho_p, rho_true)
                fidelities.append(fid)
                
                # 计算约束违反
                cv = qt.constraint_violation(rho_p)
                constraint_violations.append(cv)
    
    violation_rate = (violation_count / raw_total) if raw_total > 0 else 0.0
    
    results = {
        'fidelity_mean': np.mean(fidelities) if fidelities else None,
        'fidelity_std': np.std(fidelities) if fidelities else None,
        'mse_mean': np.mean(mses) if mses else None,
        'mse_std': np.std(mses) if mses else None,
        'cv_mean': np.mean(constraint_violations) if constraint_violations else None,
        'cv_std': np.std(constraint_violations) if constraint_violations else None,
        'violation_rate_raw': violation_rate
    }
    
    return results

## 实验执行 - 2量子比特系统

In [99]:
# 检查CUDA可用性
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 生成2量子比特数据集
n_qubits = 2
train_dataset = MultiQubitDataset(n_qubits=n_qubits, n_samples=8000, n_measurements=256, noise_level=0.02)
test_dataset = MultiQubitDataset(n_qubits=n_qubits, n_samples=2000, n_measurements=256, noise_level=0.02)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")
print(f"测量数据维度: {train_dataset.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset.target_params.shape[1]}")

# 数据验证
print("数据样本验证:")
sample_batch = next(iter(train_loader))
print(f"批次测量数据形状: {sample_batch['measurements'].shape}")
print(f"批次目标参数形状: {sample_batch['target_params'].shape}")
print(f"批次真实状态形状: {sample_batch['true_state'].shape}")

使用设备: cpu
生成 8000 个 2量子比特 量子态的数据集...


100%|██████████| 8000/8000 [00:03<00:00, 2255.44it/s]


数据集生成完成！
  测量数据形状: (8000, 16)
  真实状态形状: (8000, 4, 4)
  目标参数形状: (8000, 16)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 2000 个 2量子比特 量子态的数据集...


100%|██████████| 2000/2000 [00:00<00:00, 2041.50it/s]

数据集生成完成！
  测量数据形状: (2000, 16)
  真实状态形状: (2000, 4, 4)
  目标参数形状: (2000, 16)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
训练集大小: 8000
测试集大小: 2000
测量数据维度: 16
目标参数维度: 16
数据样本验证:
批次测量数据形状: torch.Size([32, 16])
批次目标参数形状: torch.Size([32, 16])
批次真实状态形状: torch.Size([32, 4, 4])


## 模型训练与比较

In [100]:
# 定义模型
input_dim = train_dataset.measurements.shape[1]
output_dim = train_dataset.target_params.shape[1]

models = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, output_dim)
    ),
    'PINN': MultiQubitPINN(n_qubits, input_dim, [128, 64], output_dim)
}

print("模型定义完成！")
for name, model in models.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

模型定义完成！
Traditional NN: 11472 parameters
PINN: 11856 parameters


In [101]:
# 训练参数
EPOCHS = 30
results = {}
trained_models = {}

# 训练所有模型
for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"训练 {name}")
    print(f"{'='*50}")
    
    # 创建训练器
    trainer = MultiQubitTrainer(model, device)
    
    # 训练模型
    history = trainer.train(train_loader, test_loader, epochs=EPOCHS)
    
    # 保存结果
    results[name] = history
    trained_models[name] = model
    
    # 评估模型
    eval_results = evaluate_model(model, test_loader, device)
    results[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n所有模型训练完成！")


训练 Traditional NN
开始训练 Sequential，计划 30 轮...


  3%|▎         | 1/30 [00:00<00:05,  5.35it/s]

Epoch 1/30, Train Loss: 0.0181, Val Loss: 0.0082, CV: 0.000000, LR: 1.00e-03


 20%|██        | 6/30 [00:01<00:04,  5.44it/s]

Epoch 5/30, Train Loss: 0.0059, Val Loss: 0.0059, CV: 0.000000, LR: 1.00e-03


...

 83%|████████▎ | 25/30 [00:18<00:03,  1.36it/s]

Epoch 25/30, Train Loss: 0.0034, Val Loss: 0.0023, CV: 0.000000, LR: 1.00e-03


100%|██████████| 30/30 [00:22<00:00,  1.33it/s]

Epoch 30/30, Train Loss: 0.0034, Val Loss: 0.0022, CV: 0.000000, LR: 1.00e-03



PINN 最终评估结果:
  fidelity_mean: 0.982318
  fidelity_std: 0.024160
  mse_mean: 0.002218
  mse_std: 0.000578
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

所有模型训练完成！


## 3×3维量子态系统扩展实验

In [102]:
# 生成3×3维量子态数据集（替代原来的3量子比特）
dim_3 = 3
train_dataset_3 = MultiQubitDataset(dim=dim_3, n_samples=6000, n_measurements=128, noise_level=0.03)
test_dataset_3 = MultiQubitDataset(dim=dim_3, n_samples=1500, n_measurements=128, noise_level=0.03)

# 创建数据加载器
train_loader_3 = DataLoader(train_dataset_3, batch_size=32, shuffle=True)
test_loader_3 = DataLoader(test_dataset_3, batch_size=32, shuffle=False)

print(f"3×3维量子态系统:")
print(f"训练集大小: {len(train_dataset_3)}")
print(f"测试集大小: {len(test_dataset_3)}")
print(f"测量数据维度: {train_dataset_3.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_3.target_params.shape[1]}")

生成 6000 个 3×3 量子态的数据集...


100%|██████████| 6000/6000 [00:01<00:00, 5066.51it/s]


数据集生成完成！
  测量数据形状: (6000, 9)
  真实状态形状: (6000, 3, 3)
  目标参数形状: (6000, 9)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 1500 个 3×3 量子态的数据集...


100%|██████████| 1500/1500 [00:00<00:00, 5613.94it/s]

数据集生成完成！
  测量数据形状: (1500, 9)
  真实状态形状: (1500, 3, 3)
  目标参数形状: (1500, 9)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
3×3维量子态系统:
训练集大小: 6000
测试集大小: 1500
测量数据维度: 9
目标参数维度: 9


In [103]:
# 定义3量子比特模型
input_dim_3 = train_dataset_3.measurements.shape[1]
output_dim_3 = train_dataset_3.target_params.shape[1]

models_3 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_3, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim_3)
    ),
    'PINN': MultiQubitPINN(
        dim=3,  # 3×3维量子态，使用dim参数
        input_dim=input_dim_3,
        hidden_dims=[256, 128],
        output_dim=output_dim_3,
        dropout=0.1
    )
}

# 训练3量子比特模型
results_3 = {}
trained_models_3 = {}

for name, model in models_3.items():
    print(f"\n{'='*50}")
    print(f"训练 3量子比特 {name}")
    print(f"{'='*50}")
    
    trainer = MultiQubitTrainer(model, device)
    history = trainer.train(train_loader_3, test_loader_3, epochs=20)
    
    results_3[name] = history
    trained_models_3[name] = model
    
    eval_results = evaluate_model(model, test_loader_3, device)
    results_3[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n3量子比特系统训练完成！")


训练 3量子比特 Traditional NN
开始训练 Sequential，计划 20 轮...


  5%|▌         | 1/20 [00:00<00:03,  6.30it/s]

Epoch 1/20, Train Loss: 0.0376, Val Loss: 0.0231, CV: 0.000000, LR: 1.00e-03


 30%|███       | 6/20 [00:00<00:02,  6.38it/s]

Epoch 5/20, Train Loss: 0.0193, Val Loss: 0.0183, CV: 0.000000, LR: 1.00e-03


...

Epoch 10/20, Train Loss: 0.0159, Val Loss: 0.0142, CV: 0.000000, LR: 1.00e-03


 75%|███████▌  | 15/20 [00:08<00:02,  1.87it/s]

Epoch 15/20, Train Loss: 0.0151, Val Loss: 0.0140, CV: 0.000000, LR: 1.00e-03


100%|██████████| 20/20 [00:10<00:00,  1.86it/s]

Epoch 20/20, Train Loss: 0.0143, Val Loss: 0.0133, CV: 0.000000, LR: 5.00e-04

PINN 最终评估结果:
  fidelity_mean: 0.934669
  fidelity_std: 0.080623
  mse_mean: 0.013291
  mse_std: 0.002936
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

3量子比特系统训练完成！


## 4×4维量子态系统扩展实验


In [104]:
# 生成4×4维量子态数据集（替代原来的4量子比特）
dim_4 = 4
train_dataset_4 = MultiQubitDataset(
    dim=dim_4,
    n_samples=8000,
    n_measurements=512,
    noise_level=0.03
)
test_dataset_4 = MultiQubitDataset(
    dim=dim_4,
    n_samples=1500,
    n_measurements=512,
    noise_level=0.03
)

train_loader_4 = DataLoader(train_dataset_4, batch_size=32, shuffle=True)
test_loader_4 = DataLoader(test_dataset_4, batch_size=32, shuffle=False)

print("4×4维量子态数据配置：")
print(f"训练集大小: {len(train_dataset_4)}")
print(f"测试集大小: {len(test_dataset_4)}")
print(f"测量数据维度: {train_dataset_4.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_4.target_params.shape[1]}")


生成 8000 个 4×4 量子态的数据集...


100%|██████████| 8000/8000 [00:03<00:00, 2214.44it/s]


数据集生成完成！
  测量数据形状: (8000, 16)
  真实状态形状: (8000, 4, 4)
  目标参数形状: (8000, 16)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 1500 个 4×4 量子态的数据集...


100%|██████████| 1500/1500 [00:00<00:00, 2291.70it/s]

数据集生成完成！
  测量数据形状: (1500, 16)
  真实状态形状: (1500, 4, 4)
  目标参数形状: (1500, 16)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
4×4维量子态数据配置：
训练集大小: 8000
测试集大小: 1500
测量数据维度: 16
目标参数维度: 16


In [105]:
# 定义并训练4量子比特模型（增强版）
input_dim_4 = train_dataset_4.measurements.shape[1]
output_dim_4 = train_dataset_4.target_params.shape[1]

models_4 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_4, 512),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim_4)
    ),
    'PINN': MultiQubitPINN(
        n_qubits=2,  # 4×4维量子态 = 2量子比特系统
        input_dim=input_dim_4,
        hidden_dims=[1024, 512, 512, 256, 128],
        output_dim=output_dim_4,
        dropout=0.15
    )
}

trainer_configs_4 = {
    'Traditional NN': {
        'epochs': 40,
        'lr': 1e-3,
        'use_scheduler': False,
        'grad_clip': None,
        'log_interval': 5
    },
    'PINN': {
        'epochs': 40,
        'lr': 5e-4,
        'physics_warmup': 10,
        'grad_clip': 0.8,
        'lr_factor': 0.6,
        'patience': 4,
        'min_lr': 5e-5,
        'log_interval': 5
    }
}

print("4量子比特模型定义完成！")
for name, model in models_4.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

results_4 = {}
trained_models_4 = {}

for name, model in models_4.items():
    print(f"\n{'='*50}")
    print(f"训练 4量子比特 {name}")
    print(f"{'='*50}")
    config = trainer_configs_4.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_4, test_loader_4)
    results_4[name] = history
    trained_models_4[name] = model
    eval_results = evaluate_model(model, test_loader_4, device)
    results_4[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n4量子比特系统训练完成！")

4量子比特模型定义完成！
Traditional NN: 174992 parameters
PINN: 976016 parameters

训练 4量子比特 Traditional NN
开始训练 Sequential，计划 40 轮...


  2%|▎         | 1/40 [00:00<00:15,  2.56it/s]

Epoch 1/40, Train Loss: 0.0117, Val Loss: 0.0055, CV: 0.000000, LR: 1.00e-03


 12%|█▎        | 5/40 [00:01<00:13,  2.64it/s]

Epoch 5/40, Train Loss: 0.0037, Val Loss: 0.0030, CV: 0.000000, LR: 1.00e-03


...

 88%|████████▊ | 35/40 [01:23<00:12,  2.52s/it]

Epoch 35/40, Train Loss: 0.0021, Val Loss: 0.0015, CV: 0.000000, LR: 5.00e-04


100%|██████████| 40/40 [01:36<00:00,  2.41s/it]

Epoch 40/40, Train Loss: 0.0019, Val Loss: 0.0011, CV: 0.000000, LR: 3.00e-04



PINN 最终评估结果:
  fidelity_mean: 0.992659
  fidelity_std: 0.015405
  mse_mean: 0.001130
  mse_std: 0.000926
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

4量子比特系统训练完成！


## 5×5维量子态系统扩展实验


In [106]:
# 生成5×5维量子态数据集（替代原来的5量子比特）
dim_5 = 5
train_dataset_5 = MultiQubitDataset(dim=dim_5, n_samples=3000, n_measurements=32, noise_level=0.05)
test_dataset_5 = MultiQubitDataset(dim=dim_5, n_samples=800, n_measurements=32, noise_level=0.05)

# 创建数据加载器（进一步减小batch size以适应更大的参数空间）
train_loader_5 = DataLoader(train_dataset_5, batch_size=8, shuffle=True)
test_loader_5 = DataLoader(test_dataset_5, batch_size=8, shuffle=False)

print(f"5×5维量子态系统:")
print(f"训练集大小: {len(train_dataset_5)}")
print(f"测试集大小: {len(test_dataset_5)}")
print(f"测量数据维度: {train_dataset_5.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_5.target_params.shape[1]}")


生成 3000 个 5×5 量子态的数据集...


100%|██████████| 3000/3000 [00:01<00:00, 2193.82it/s]


数据集生成完成！
  测量数据形状: (3000, 25)
  真实状态形状: (3000, 5, 5)
  目标参数形状: (3000, 25)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 800 个 5×5 量子态的数据集...


100%|██████████| 800/800 [00:00<00:00, 2193.63it/s]

数据集生成完成！
  测量数据形状: (800, 25)
  真实状态形状: (800, 5, 5)
  目标参数形状: (800, 25)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
5×5维量子态系统:
训练集大小: 3000
测试集大小: 800
测量数据维度: 25
目标参数维度: 25


In [107]:
# 定义5量子比特模型
input_dim_5 = train_dataset_5.measurements.shape[1]
output_dim_5 = train_dataset_5.target_params.shape[1]

models_5 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_5, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim_5)
    ),
    'PINN': MultiQubitPINN(
        dim=5,  # 5×5维量子态，使用dim参数
        input_dim=input_dim_5,
        hidden_dims=[1024, 512, 256, 128],
        output_dim=output_dim_5,
        dropout=0.15
    )
}

print("5量子比特模型定义完成！")
for name, model in models_5.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练5量子比特模型
results_5 = {}
trained_models_5 = {}

for name, model in models_5.items():
    print(f"\n{'='*50}")
    print(f"训练 5量子比特 {name}")
    print(f"{'='*50}")
    
    trainer = MultiQubitTrainer(model, device)
    history = trainer.train(train_loader_5, test_loader_5, epochs=12)
    
    results_5[name] = history
    trained_models_5[name] = model
    
    eval_results = evaluate_model(model, test_loader_5, device)
    results_5[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n5量子比特系统训练完成！")


5量子比特模型定义完成！
Traditional NN: 718873 parameters
PINN: 722713 parameters

训练 5量子比特 Traditional NN
开始训练 Sequential，计划 12 轮...


  8%|▊         | 1/12 [00:01<00:14,  1.35s/it]

Epoch 1/12, Train Loss: 0.0200, Val Loss: 0.0179, CV: 0.000000, LR: 1.00e-03


 42%|████▏     | 5/12 [00:06<00:09,  1.31s/it]

Epoch 5/12, Train Loss: 0.0164, Val Loss: 0.0174, CV: 0.000000, LR: 1.00e-03


...

Epoch 5/12, Train Loss: 0.0167, Val Loss: 0.0163, CV: 0.000000, LR: 1.00e-03


 83%|████████▎ | 10/12 [00:28<00:05,  2.90s/it]

Epoch 10/12, Train Loss: 0.0154, Val Loss: 0.0172, CV: 0.000000, LR: 1.00e-03


100%|██████████| 12/12 [00:33<00:00,  2.76s/it]



PINN 最终评估结果:
  fidelity_mean: 0.684012
  fidelity_std: 0.308213
  mse_mean: 0.016144
  mse_std: 0.003135
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

5量子比特系统训练完成！


## 6×6维量子态系统扩展实验


In [108]:
# 生成6×6维量子态数据集
dim_6 = 6
train_dataset_6 = MultiQubitDataset(dim=dim_6, n_samples=2000, n_measurements=64, noise_level=0.05)
test_dataset_6 = MultiQubitDataset(dim=dim_6, n_samples=500, n_measurements=64, noise_level=0.05)

# 创建数据加载器（减小batch size以适应更大的参数空间）
train_loader_6 = DataLoader(train_dataset_6, batch_size=8, shuffle=True)
test_loader_6 = DataLoader(test_dataset_6, batch_size=8, shuffle=False)

print(f"6×6维量子态系统:")
print(f"训练集大小: {len(train_dataset_6)}")
print(f"测试集大小: {len(test_dataset_6)}")
print(f"测量数据维度: {train_dataset_6.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_6.target_params.shape[1]}")


生成 2000 个 6×6 量子态的数据集...


100%|██████████| 2000/2000 [00:01<00:00, 1550.67it/s]


数据集生成完成！
  测量数据形状: (2000, 36)
  真实状态形状: (2000, 6, 6)
  目标参数形状: (2000, 36)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 500 个 6×6 量子态的数据集...


100%|██████████| 500/500 [00:00<00:00, 1562.66it/s]

数据集生成完成！
  测量数据形状: (500, 36)
  真实状态形状: (500, 6, 6)
  目标参数形状: (500, 36)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
6×6维量子态系统:
训练集大小: 2000
测试集大小: 500
测量数据维度: 36
目标参数维度: 36


In [109]:
# 定义6×6维量子态模型
input_dim_6 = train_dataset_6.measurements.shape[1]
output_dim_6 = train_dataset_6.target_params.shape[1]

models_6 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_6, 1024),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim_6)
    ),
    'PINN': MultiQubitPINN(
        dim=dim_6,  # 6×6维量子态，使用dim参数
        input_dim=input_dim_6,
        hidden_dims=[1024, 512, 256, 128],
        output_dim=output_dim_6,
        dropout=0.15
    )
}

print("6×6维量子态模型定义完成！")
for name, model in models_6.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练6×6维量子态模型
results_6 = {}
trained_models_6 = {}

trainer_configs_6 = {
    'Traditional NN': {
        'epochs': 15,
        'lr': 1e-3,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'log_interval': 3
    },
    'PINN': {
        'epochs': 15,
        'lr': 5e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'physics_warmup': 5,
        'log_interval': 3
    }
}

for name, model in models_6.items():
    print(f"\n{'='*50}")
    print(f"训练 6×6维量子态 {name}")
    print(f"{'='*50}")
    
    config = trainer_configs_6.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_6, test_loader_6)
    
    results_6[name] = history
    trained_models_6[name] = model
    
    eval_results = evaluate_model(model, test_loader_6, device)
    results_6[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n6×6维量子态系统训练完成！")


6×6维量子态模型定义完成！
Traditional NN: 731556 parameters
PINN: 735396 parameters

训练 6×6维量子态 Traditional NN
开始训练 Sequential，计划 15 轮...


  7%|▋         | 1/15 [00:01<00:15,  1.13s/it]

Epoch 1/15, Train Loss: 0.0150, Val Loss: 0.0125, CV: 0.000000, LR: 1.00e-03


 20%|██        | 3/15 [00:03<00:13,  1.10s/it]

Epoch 3/15, Train Loss: 0.0122, Val Loss: 0.0120, CV: 0.000000, LR: 1.00e-03


...

Epoch 9/15, Train Loss: 0.0104, Val Loss: 0.0106, CV: 0.000000, LR: 5.00e-04


 80%|████████  | 12/15 [00:20<00:05,  1.74s/it]

Epoch 12/15, Train Loss: 0.0095, Val Loss: 0.0111, CV: 0.000000, LR: 5.00e-04


100%|██████████| 15/15 [00:26<00:00,  1.75s/it]

Epoch 15/15, Train Loss: 0.0089, Val Loss: 0.0111, CV: 0.000000, LR: 5.00e-04

PINN 最终评估结果:
  fidelity_mean: 0.686311
  fidelity_std: 0.299310
  mse_mean: 0.011099
  mse_std: 0.002314
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

6×6维量子态系统训练完成！


## 7×7维量子态系统扩展实验


In [110]:
# 生成7×7维量子态数据集
dim_7 = 7
train_dataset_7 = MultiQubitDataset(dim=dim_7, n_samples=1500, n_measurements=64, noise_level=0.05)
test_dataset_7 = MultiQubitDataset(dim=dim_7, n_samples=400, n_measurements=64, noise_level=0.05)

# 创建数据加载器（进一步减小batch size）
train_loader_7 = DataLoader(train_dataset_7, batch_size=4, shuffle=True)
test_loader_7 = DataLoader(test_dataset_7, batch_size=4, shuffle=False)

print(f"7×7维量子态系统:")
print(f"训练集大小: {len(train_dataset_7)}")
print(f"测试集大小: {len(test_dataset_7)}")
print(f"测量数据维度: {train_dataset_7.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_7.target_params.shape[1]}")


生成 1500 个 7×7 量子态的数据集...


100%|██████████| 1500/1500 [00:01<00:00, 1173.11it/s]


数据集生成完成！
  测量数据形状: (1500, 49)
  真实状态形状: (1500, 7, 7)
  目标参数形状: (1500, 49)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 400 个 7×7 量子态的数据集...


100%|██████████| 400/400 [00:00<00:00, 1172.15it/s]

数据集生成完成！
  测量数据形状: (400, 49)
  真实状态形状: (400, 7, 7)
  目标参数形状: (400, 49)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
7×7维量子态系统:
训练集大小: 1500
测试集大小: 400
测量数据维度: 49
目标参数维度: 49


In [111]:
# 定义7×7维量子态模型
input_dim_7 = train_dataset_7.measurements.shape[1]
output_dim_7 = train_dataset_7.target_params.shape[1]

models_7 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_7, 1024),
        nn.ReLU(),
        nn.Dropout(0.15),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.15),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, output_dim_7)
    ),
    'PINN': MultiQubitPINN(
        dim=dim_7,  # 7×7维量子态，使用dim参数
        input_dim=input_dim_7,
        hidden_dims=[1024, 512, 256],
        output_dim=output_dim_7,
        dropout=0.15
    )
}

print("7×7维量子态模型定义完成！")
for name, model in models_7.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练7×7维量子态模型
results_7 = {}
trained_models_7 = {}

trainer_configs_7 = {
    'Traditional NN': {
        'epochs': 12,
        'lr': 8e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'log_interval': 3
    },
    'PINN': {
        'epochs': 12,
        'lr': 5e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'physics_warmup': 5,
        'log_interval': 3
    }
}

for name, model in models_7.items():
    print(f"\n{'='*50}")
    print(f"训练 7×7维量子态 {name}")
    print(f"{'='*50}")
    
    config = trainer_configs_7.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_7, test_loader_7)
    
    results_7[name] = history
    trained_models_7[name] = model
    
    eval_results = evaluate_model(model, test_loader_7, device)
    results_7[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n7×7维量子态系统训练完成！")


7×7维量子态模型定义完成！
Traditional NN: 719921 parameters
PINN: 723505 parameters

训练 7×7维量子态 Traditional NN
开始训练 Sequential，计划 12 轮...


  8%|▊         | 1/12 [00:01<00:15,  1.40s/it]

Epoch 1/12, Train Loss: 0.0126, Val Loss: 0.0106, CV: 0.000000, LR: 8.00e-04


 25%|██▌       | 3/12 [00:04<00:12,  1.42s/it]

Epoch 3/12, Train Loss: 0.0097, Val Loss: 0.0092, CV: 0.000000, LR: 8.00e-04


...

Epoch 6/12, Train Loss: 0.0091, Val Loss: 0.0103, CV: 0.000000, LR: 5.00e-04


 75%|███████▌  | 9/12 [00:21<00:07,  2.35s/it]

Epoch 9/12, Train Loss: 0.0086, Val Loss: 0.0086, CV: 0.000000, LR: 5.00e-04


100%|██████████| 12/12 [00:28<00:00,  2.36s/it]

Epoch 12/12, Train Loss: 0.0080, Val Loss: 0.0088, CV: 0.000000, LR: 5.00e-04

PINN 最终评估结果:
  fidelity_mean: 0.643551
  fidelity_std: 0.313833
  mse_mean: 0.008845
  mse_std: 0.001673
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

7×7维量子态系统训练完成！


## 8×8维量子态系统扩展实验


In [112]:
# 生成8×8维量子态数据集
dim_8 = 8
train_dataset_8 = MultiQubitDataset(dim=dim_8, n_samples=1500, n_measurements=64, noise_level=0.05)
test_dataset_8 = MultiQubitDataset(dim=dim_8, n_samples=400, n_measurements=64, noise_level=0.05)

# 创建数据加载器（进一步减小batch size）
train_loader_8 = DataLoader(train_dataset_8, batch_size=4, shuffle=True)
test_loader_8 = DataLoader(test_dataset_8, batch_size=4, shuffle=False)

print(f"8×8维量子态系统:")
print(f"训练集大小: {len(train_dataset_8)}")
print(f"测试集大小: {len(test_dataset_8)}")
print(f"测量数据维度: {train_dataset_8.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_8.target_params.shape[1]}")


生成 1500 个 8×8 量子态的数据集...


100%|██████████| 1500/1500 [00:03<00:00, 465.00it/s]


数据集生成完成！
  测量数据形状: (1500, 64)
  真实状态形状: (1500, 8, 8)
  目标参数形状: (1500, 64)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 400 个 8×8 量子态的数据集...


100%|██████████| 400/400 [00:00<00:00, 454.03it/s]

数据集生成完成！
  测量数据形状: (400, 64)
  真实状态形状: (400, 8, 8)
  目标参数形状: (400, 64)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
8×8维量子态系统:
训练集大小: 1500
测试集大小: 400
测量数据维度: 64
目标参数维度: 64


In [113]:
# 定义8×8维量子态模型
# 8×8是2的幂次（8=2³），可以使用n_qubits或dim参数
input_dim_8 = train_dataset_8.measurements.shape[1]
output_dim_8 = train_dataset_8.target_params.shape[1]
n_qubits_8 = 3  # 8 = 2^3

models_8 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_8, 1024),
        nn.ReLU(),
        nn.Dropout(0.15),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.15),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, output_dim_8)
    ),
    'PINN': MultiQubitPINN(
        n_qubits=n_qubits_8,  # 可以使用n_qubits或dim
        input_dim=input_dim_8,
        hidden_dims=[1024, 512, 256],
        output_dim=output_dim_8,
        dropout=0.15
    )
}

print("8×8维量子态模型定义完成！")
for name, model in models_8.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练8×8维量子态模型
results_8 = {}
trained_models_8 = {}

trainer_configs_8 = {
    'Traditional NN': {
        'epochs': 12,
        'lr': 8e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'log_interval': 3
    },
    'PINN': {
        'epochs': 12,
        'lr': 5e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'physics_warmup': 5,
        'log_interval': 3
    }
}

for name, model in models_8.items():
    print(f"\n{'='*50}")
    print(f"训练 8×8维量子态 {name}")
    print(f"{'='*50}")
    
    config = trainer_configs_8.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_8, test_loader_8)
    
    results_8[name] = history
    trained_models_8[name] = model
    
    eval_results = evaluate_model(model, test_loader_8, device)
    results_8[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n8×8维量子态系统训练完成！")


8×8维量子态模型定义完成！
Traditional NN: 739136 parameters
PINN: 742720 parameters

训练 8×8维量子态 Traditional NN
开始训练 Sequential，计划 12 轮...


  8%|▊         | 1/12 [00:01<00:15,  1.44s/it]

Epoch 1/12, Train Loss: 0.0069, Val Loss: 0.0060, CV: 0.000000, LR: 8.00e-04


 25%|██▌       | 3/12 [00:04<00:13,  1.45s/it]

Epoch 3/12, Train Loss: 0.0048, Val Loss: 0.0045, CV: 0.000000, LR: 8.00e-04


...

Epoch 6/12, Train Loss: 0.0034, Val Loss: 0.0034, CV: 0.000000, LR: 5.00e-04


 75%|███████▌  | 9/12 [00:26<00:08,  2.93s/it]

Epoch 9/12, Train Loss: 0.0027, Val Loss: 0.0031, CV: 0.000000, LR: 5.00e-04


100%|██████████| 12/12 [00:34<00:00,  2.88s/it]

Epoch 12/12, Train Loss: 0.0023, Val Loss: 0.0030, CV: 0.000000, LR: 5.00e-04

PINN 最终评估结果:
  fidelity_mean: 0.856491
  fidelity_std: 0.178770
  mse_mean: 0.002956
  mse_std: 0.001184
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

8×8维量子态系统训练完成！


## 9×9维量子态系统扩展实验


In [122]:
# 生成9×9维量子态数据集
dim_9 = 9
train_dataset_9 = MultiQubitDataset(dim=dim_9, n_samples=1000, n_measurements=64, noise_level=0.05)
test_dataset_9 = MultiQubitDataset(dim=dim_9, n_samples=250, n_measurements=64, noise_level=0.05)

# 创建数据加载器（进一步减小batch size）
train_loader_9 = DataLoader(train_dataset_9, batch_size=4, shuffle=True)
test_loader_9 = DataLoader(test_dataset_9, batch_size=4, shuffle=False)

print(f"9×9维量子态系统:")
print(f"训练集大小: {len(train_dataset_9)}")
print(f"测试集大小: {len(test_dataset_9)}")
print(f"测量数据维度: {train_dataset_9.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_9.target_params.shape[1]}")


生成 1000 个 9×9 量子态的数据集...


100%|██████████| 1000/1000 [00:01<00:00, 648.03it/s]


数据集生成完成！
  测量数据形状: (1000, 81)
  真实状态形状: (1000, 9, 9)
  目标参数形状: (1000, 81)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 250 个 9×9 量子态的数据集...


100%|██████████| 250/250 [00:00<00:00, 691.30it/s]

数据集生成完成！
  测量数据形状: (250, 81)
  真实状态形状: (250, 9, 9)
  目标参数形状: (250, 81)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
9×9维量子态系统:
训练集大小: 1000
测试集大小: 250
测量数据维度: 81
目标参数维度: 81


In [121]:
# 定义9×9维量子态模型
input_dim_9 = train_dataset_9.measurements.shape[1]
output_dim_9 = train_dataset_9.target_params.shape[1]

models_9 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_9, 1024),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, output_dim_9)
    ),
    'PINN': MultiQubitPINN(
        dim=dim_9,  # 9×9维量子态，使用dim参数
        input_dim=input_dim_9,
        hidden_dims=[1024, 512, 256],
        output_dim=output_dim_9,
        dropout=0.2
    )
}

print("9×9维量子态模型定义完成！")
for name, model in models_9.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练9×9维量子态模型
results_9 = {}
trained_models_9 = {}

trainer_configs_9 = {
    'Traditional NN': {
        'epochs': 15,
        'lr': 5e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'log_interval': 3
    },
    'PINN': {
        'epochs': 18,  # 增加训练轮数，让物理约束有更多时间发挥作用
        'lr': 4e-4,  # 稍微降低学习率，更稳定
        'use_scheduler': True,
        'grad_clip': 1.0,
        'physics_warmup': 8,  # 增加warmup轮数，让物理约束逐渐增强
        'log_interval': 3
    }
}

for name, model in models_9.items():
    print(f"\n{'='*50}")
    print(f"训练 9×9维量子态 {name}")
    print(f"{'='*50}")
    
    config = trainer_configs_9.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_9, test_loader_9)
    
    results_9[name] = history
    trained_models_9[name] = model
    
    eval_results = evaluate_model(model, test_loader_9, device)
    results_9[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n9×9维量子态系统训练完成！")


9×9维量子态模型定义完成！
Traditional NN: 760913 parameters
PINN: 764497 parameters

训练 9×9维量子态 Traditional NN
开始训练 Sequential，计划 15 轮...


  7%|▋         | 1/15 [00:01<00:14,  1.03s/it]

Epoch 1/15, Train Loss: 0.0086, Val Loss: 0.0076, CV: 0.000000, LR: 5.00e-04


 20%|██        | 3/15 [00:03<00:13,  1.15s/it]

Epoch 3/15, Train Loss: 0.0066, Val Loss: 0.0063, CV: 0.000000, LR: 5.00e-04


...

Epoch 12/18, Train Loss: 0.0059, Val Loss: 0.0058, CV: 0.000000, LR: 4.00e-04


 83%|████████▎ | 15/18 [00:30<00:05,  1.95s/it]

Epoch 15/18, Train Loss: 0.0057, Val Loss: 0.0055, CV: 0.000000, LR: 4.00e-04


100%|██████████| 18/18 [00:36<00:00,  2.01s/it]

Epoch 18/18, Train Loss: 0.0054, Val Loss: 0.0055, CV: 0.000000, LR: 4.00e-04

PINN 最终评估结果:
  fidelity_mean: 0.620442
  fidelity_std: 0.362452
  mse_mean: 0.005468
  mse_std: 0.001024
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

9×9维量子态系统训练完成！


## 10×10维量子态系统扩展实验


In [116]:
# 生成10×10维量子态数据集
dim_10 = 10
train_dataset_10 = MultiQubitDataset(dim=dim_10, n_samples=800, n_measurements=64, noise_level=0.05)
test_dataset_10 = MultiQubitDataset(dim=dim_10, n_samples=200, n_measurements=64, noise_level=0.05)

# 创建数据加载器（使用最小batch size）
train_loader_10 = DataLoader(train_dataset_10, batch_size=2, shuffle=True)
test_loader_10 = DataLoader(test_dataset_10, batch_size=2, shuffle=False)

print(f"10×10维量子态系统:")
print(f"训练集大小: {len(train_dataset_10)}")
print(f"测试集大小: {len(test_dataset_10)}")
print(f"测量数据维度: {train_dataset_10.measurements.shape[1]}")
print(f"目标参数维度: {train_dataset_10.target_params.shape[1]}")


生成 800 个 10×10 量子态的数据集...


100%|██████████| 800/800 [00:01<00:00, 576.14it/s]


数据集生成完成！
  测量数据形状: (800, 100)
  真实状态形状: (800, 10, 10)
  目标参数形状: (800, 100)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
生成 200 个 10×10 量子态的数据集...


100%|██████████| 200/200 [00:00<00:00, 572.81it/s]

数据集生成完成！
  测量数据形状: (200, 100)
  真实状态形状: (200, 10, 10)
  目标参数形状: (200, 100)
  状态类型示例: ['GHZ' 'RandomMixed' 'RandomPure' 'W']
10×10维量子态系统:
训练集大小: 800
测试集大小: 200
测量数据维度: 100
目标参数维度: 100


In [125]:
# 定义10×10维量子态模型
input_dim_10 = train_dataset_10.measurements.shape[1]
output_dim_10 = train_dataset_10.target_params.shape[1]

models_10 = {
    'Traditional NN': nn.Sequential(
        nn.Linear(input_dim_10, 1024),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, output_dim_10)
    ),
    'PINN': MultiQubitPINN(
        dim=dim_10,  # 10×10维量子态，使用dim参数
        input_dim=input_dim_10,
        hidden_dims=[1536, 1024, 512, 256],  # 增加网络深度和宽度
        output_dim=output_dim_10,
        dropout=0.25  # 增加dropout防止过拟合
    )
}

print("10×10维量子态模型定义完成！")
for name, model in models_10.items():
    print(f"{name}: {sum(p.numel() for p in model.parameters())} parameters")

# 训练10×10维量子态模型
results_10 = {}
trained_models_10 = {}

trainer_configs_10 = {
    'Traditional NN': {
        'epochs': 15,
        'lr': 5e-4,
        'use_scheduler': True,
        'grad_clip': 1.0,
        'log_interval': 3
    },
    'PINN': {
        'epochs': 40,  # 大幅增加训练轮数到40轮
        'lr': 2.5e-4,  # 进一步降低学习率，更稳定
        'use_scheduler': True,
        'lr_factor': 0.8,  # 学习率衰减因子
        'patience': 6,  # 更耐心的学习率调度
        'grad_clip': 0.5,  # 减小梯度裁剪，允许更大的更新
        'physics_warmup': 15,  # 增加warmup到15轮，让物理约束充分预热
        'log_interval': 5
    }
}

for name, model in models_10.items():
    print(f"\n{'='*50}")
    print(f"训练 10×10维量子态 {name}")
    print(f"{'='*50}")
    
    config = trainer_configs_10.get(name, {})
    trainer = MultiQubitTrainer(model, device, config=config)
    history = trainer.train(train_loader_10, test_loader_10)
    
    results_10[name] = history
    trained_models_10[name] = model
    
    eval_results = evaluate_model(model, test_loader_10, device)
    results_10[name].update(eval_results)
    
    print(f"\n{name} 最终评估结果:")
    for key, value in eval_results.items():
        if value is not None:
            print(f"  {key}: {value:.6f}")

print("\n10×10维量子态系统训练完成！")


10×10维量子态模型定义完成！
Traditional NN: 785252 parameters
PINN: 2417508 parameters

训练 10×10维量子态 Traditional NN
开始训练 Sequential，计划 15 轮...


  7%|▋         | 1/15 [00:01<00:25,  1.80s/it]

Epoch 1/15, Train Loss: 0.0072, Val Loss: 0.0070, CV: 0.000000, LR: 5.00e-04


 20%|██        | 3/15 [00:05<00:20,  1.71s/it]

Epoch 3/15, Train Loss: 0.0061, Val Loss: 0.0065, CV: 0.000000, LR: 5.00e-04


...

 88%|████████▊ | 35/40 [03:17<00:29,  5.81s/it]

Epoch 35/40, Train Loss: 0.0044, Val Loss: 0.0050, CV: 0.000000, LR: 1.60e-04


100%|██████████| 40/40 [03:46<00:00,  5.66s/it]

Epoch 40/40, Train Loss: 0.0041, Val Loss: 0.0050, CV: 0.000000, LR: 1.60e-04



PINN 最终评估结果:
  fidelity_mean: 0.580192
  fidelity_std: 0.399490
  mse_mean: 0.005026
  mse_std: 0.004612
  cv_mean: 0.000000
  cv_std: 0.000000
  violation_rate_raw: 0.000000

10×10维量子态系统训练完成！


## 应用示例：量子纠缠健康监测

依托PINN重建得到的密度矩阵，我们可以即时评估量子设备在生产GHZ/W等目标纠缠态时是否“偏离健康状态”。
核心步骤：
- 使用训练好的PINN模型在线重建密度矩阵；
- 计算与目标标准态的保真度，并根据阈值触发告警；
- 输出纠缠态类别判别和综合健康评分，可直接反馈给实验控制系统，实现“表征→诊断→调参”的闭环。
